In [1]:
# Step 1.1: Install core Hugging Face libraries
# This installs transformers, datasets, and accelerate
!pip install transformers datasets accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 4.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 89.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 77.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 79.9 MB/s eta 0:00:00:00:0100:01
  Attempting un

In [2]:
# Step 1.3: Install the Hugging Face Evaluate library
# This library is used for evaluating the model's performance
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.7 MB/s eta 0:00:00a 0:00:01


# Import necessary Python modules

In [3]:
# Step 1.4: Import necessary Python modules
import torch                      # Core PyTorch library for tensors and neural networks
import numpy as np                # For numerical operations, especially with arrays
from datasets import load_dataset # To load datasets from Hugging Face
import evaluate                   # *** CORRECTED: Import the 'evaluate' library itself ***
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer # Key classes for BERT, tokenization, and training
import gradio as gr               # For creating the interactive web demo

print("All necessary modules imported successfully! Step 1 should now be complete.")

2025-07-28 17:02:31.452753: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753722151.836361      80 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753722151.938706      80 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


All necessary modules imported successfully! Step 1 should now be complete.


# Load the AG News dataset

In [5]:
# Step 2.1: Load the AG News dataset
print("Loading AG News dataset (full version first)...")
dataset = load_dataset("ag_news")
print("Full dataset loaded successfully!")

# Let's peek at the dataset structure to understand what we've loaded
print("\nFull Dataset structure:")
print(dataset)

Loading AG News dataset (full version first)...


README.md: 0.00B [00:00, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/18.6M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/1.23M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

Full dataset loaded successfully!

Full Dataset structure:
DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 120000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 7600
    })
})


# Load the Tokenizer

In [6]:
# Step 2.2: Load the Tokenizer
from transformers import AutoTokenizer

model_name = "bert-base-uncased" # Sticking with BERT-base for now, remember DistilBERT if VRAM is still an issue

print(f"\nLoading tokenizer for {model_name}...")
tokenizer = AutoTokenizer.from_pretrained(model_name)
print("Tokenizer loaded!")


Loading tokenizer for bert-base-uncased...


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Tokenizer loaded!


# Sample a Smaller Dataset

In [8]:
# Step 2.3: Sample a Smaller Dataset
train_size = 10000
eval_size = 1000

print(f"\nCreating smaller dataset subsets: Training={train_size}, Evaluation={eval_size}...")

# Create a smaller training dataset
# .shuffle(seed=42) ensures we get a random but reproducible subset
# .select(range(train_size)) takes the first 'train_size' examples after shuffling
small_train_dataset = dataset["train"].shuffle(seed=42).select(range(train_size))

# Create a smaller evaluation dataset
small_eval_dataset = dataset["test"].shuffle(seed=42).select(range(eval_size))

print("Smaller datasets created successfully!")

# Let's inspect the size of our new datasets
print(f"Size of small_train_dataset: {len(small_train_dataset)} samples")
print(f"Size of small_eval_dataset: {len(small_eval_dataset)} samples")

# And show a sample to confirm structure
print("\nFirst example from small_train_dataset:")
print(small_train_dataset[0])


Creating smaller dataset subsets: Training=10000, Evaluation=1000...
Smaller datasets created successfully!
Size of small_train_dataset: 10000 samples
Size of small_eval_dataset: 1000 samples

First example from small_train_dataset:
{'text': 'Bangladesh paralysed by strikes Opposition activists have brought many towns and cities in Bangladesh to a halt, the day after 18 people died in explosions at a political rally.', 'label': 0}


# Define Tokenization Function

In [9]:
# Step 2.4: Define Tokenization Function
def tokenize_function(examples):
    # The tokenizer processes the 'text' column from our dataset
    return tokenizer(examples["text"], padding=True, truncation=True)

print("Tokenization function 'tokenize_function' defined.")

Tokenization function 'tokenize_function' defined.


# Apply Tokenization to Smaller Datasets

In [10]:
# Step 2.5: Apply Tokenization to Smaller Datasets
print("\nApplying tokenization to the smaller datasets (this will be quick!)...")

# Apply the tokenization function to the small training dataset
tokenized_small_train_dataset = small_train_dataset.map(tokenize_function, batched=True)

# Apply the tokenization function to the small evaluation dataset
tokenized_small_eval_dataset = small_eval_dataset.map(tokenize_function, batched=True)

print("Tokenization of smaller datasets complete!")

# Let's inspect the structure of the tokenized smaller datasets
print("\nTokenized Small Training Dataset structure:")
print(tokenized_small_train_dataset)
print("\nFirst example from tokenized small training dataset:")
print(tokenized_small_train_dataset[0])


Applying tokenization to the smaller datasets (this will be quick!)...


Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Tokenization of smaller datasets complete!

Tokenized Small Training Dataset structure:
Dataset({
    features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 10000
})

First example from tokenized small training dataset:
{'text': 'Bangladesh paralysed by strikes Opposition activists have brought many towns and cities in Bangladesh to a halt, the day after 18 people died in explosions at a political rally.', 'label': 0, 'input_ids': [101, 7269, 11498, 2135, 6924, 2011, 9326, 4559, 10134, 2031, 2716, 2116, 4865, 1998, 3655, 1999, 7269, 2000, 1037, 9190, 1010, 1996, 2154, 2044, 2324, 2111, 2351, 1999, 18217, 2012, 1037, 2576, 8320, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

# Final Dataset Preparation for Trainer

In [11]:
# Step 2.6: Final Dataset Preparation for Trainer
print("\nPreparing final datasets for the Trainer...")

# Remove original 'text' column and rename 'label' to 'labels'
# The 'idx' column is NOT present by default in AG News, so we remove it from the list
columns_to_remove = ["text"]
tokenized_small_train_dataset = tokenized_small_train_dataset.remove_columns(columns_to_remove)
tokenized_small_eval_dataset = tokenized_small_eval_dataset.remove_columns(columns_to_remove)

# Rename 'label' to 'labels' as required by the Trainer
tokenized_small_train_dataset = tokenized_small_train_dataset.rename_column("label", "labels")
tokenized_small_eval_dataset = tokenized_small_eval_dataset.rename_column("label", "labels")

# Set the format to PyTorch tensors
tokenized_small_train_dataset.set_format("torch")
tokenized_small_eval_dataset.set_format("torch")

print("Final datasets prepared for Trainer!")

# Confirm the new structure of the datasets
print("\nFinal structure of tokenized_small_train_dataset:")
print(tokenized_small_train_dataset)
print("\nFirst example from final tokenized_small_train_dataset (showing tensor format):")
print(tokenized_small_train_dataset[0])


Preparing final datasets for the Trainer...
Final datasets prepared for Trainer!

Final structure of tokenized_small_train_dataset:
Dataset({
    features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 10000
})

First example from final tokenized_small_train_dataset (showing tensor format):
{'labels': tensor(0), 'input_ids': tensor([  101,  7269, 11498,  2135,  6924,  2011,  9326,  4559, 10134,  2031,
         2716,  2116,  4865,  1998,  3655,  1999,  7269,  2000,  1037,  9190,
         1010,  1996,  2154,  2044,  2324,  2111,  2351,  1999, 18217,  2012,
         1037,  2576,  8320,  1012,   102,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     

# Understanding Labels and Loading the Pre-trained Model

In [12]:
# Step 3.1: Understanding Labels and Loading the Pre-trained Model
from transformers import AutoModelForSequenceClassification
import numpy as np

# AG News has 4 classes: World, Sports, Business, Sci/Tech
# The labels are usually 0, 1, 2, 3
num_labels = 4

# Create mappings from ID to label name and vice versa for readability
id2label = {0: "World", 1: "Sports", 2: "Business", 3: "Sci/Tech"}
label2id = {"World": 0, "Sports": 1, "Business": 2, "Sci/Tech": 3}

print(f"\nLoading {model_name} for sequence classification with {num_labels} labels...")

# Load the pre-trained BERT model with a classification head for 4 labels
# We pass id2label and label2id to the model's configuration for better output interpretation
model = AutoModelForSequenceClassification.from_pretrained(
    model_name, num_labels=num_labels, id2label=id2label, label2id=label2id
)
print(f"Model {model_name} loaded successfully!")

# You'll likely see a warning about some weights not being initialized.
# This is normal! The pre-trained model's original classification head is discarded,
# and a new one is randomly initialized for your specific task (4 labels in our case).
# This new head is what will be trained during fine-tuning.


Loading bert-base-uncased for sequence classification with 4 labels...


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model bert-base-uncased loaded successfully!


# Define Training Arguments

In [14]:
# Step 3.2: Define Training Arguments
from transformers import TrainingArguments

print("\nSetting up Training Arguments")
training_args = TrainingArguments(
    output_dir="./results",               # Directory where the trained model and checkpoints will be saved
    eval_strategy="epoch",          # Evaluate after each training epoch
    learning_rate=2e-5,                   # Learning rate for the optimizer
    per_device_train_batch_size=16,        # Optimized batch size for RAM management
    per_device_eval_batch_size=16,         # Optimized batch size for RAM management
    num_train_epochs=1,                   # <<< Set to 1 epoch for fastest training time
    weight_decay=0.01,                    # L2 regularization to prevent overfitting
    save_strategy="epoch",                # Save a model checkpoint after each epoch
    load_best_model_at_end=True,          # After training, load the best performing model based on evaluation metric
    metric_for_best_model="f1",           # Metric to use for determining the "best" model
    report_to="none",                     # Prevents reporting to external services like Weights & Biases if not configured
)
print("Training Arguments set")


Setting up Training Arguments
Training Arguments set


# Defining compute_metrics

In [15]:
# Step 3.3: Defining compute_metrics
import evaluate # Import the evaluate library
import numpy as np # For numerical operations, especially argmax

# Load the metrics we want to compute
accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")

def compute_metrics(eval_pred):
    """
    Computes accuracy and F1 score from the model's predictions.
    Args:
        eval_pred (tuple): A tuple containing predictions (logits) and true labels.
    Returns:
        dict: A dictionary of computed metrics.
    """
    logits, labels = eval_pred
    # Get the predicted class by finding the index of the highest logit
    predictions = np.argmax(logits, axis=-1)

    # Compute accuracy
    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)

    # Compute F1 score (using 'weighted' average for multi-class to account for imbalance)
    f1 = f1_metric.compute(predictions=predictions, references=labels, average="weighted")

    # Return a dictionary of the results
    return {"accuracy": accuracy["accuracy"], "f1": f1["f1"]}

print("compute_metrics function defined.")

compute_metrics function defined.


# Initializing the Hugging Face Trainer

In [16]:
# Step 3.4: Initializing the Hugging Face Trainer
from transformers import Trainer
from transformers import DataCollatorWithPadding # Import DataCollatorWithPadding

# A DataCollator is used to form batches by padding sequences to the longest length in that batch.
# This makes training more efficient.
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

print("\nInitializing Hugging Face Trainer...")
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_small_train_dataset, # Our smaller training dataset
    eval_dataset=tokenized_small_eval_dataset,   # Our smaller evaluation dataset
    tokenizer=tokenizer,
    data_collator=data_collator,                  # Use the data collator
    compute_metrics=compute_metrics,
)
print("Trainer initialized successfully!")


Initializing Hugging Face Trainer...


/tmp/ipykernel_80/3318022645.py:10: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Trainer initialized successfully!


# Start the Model Training

In [17]:
# Step 3.5: Start the Model Training
print("\nStarting model training...")
# This call executes the training loop
trainer.train()
print("Model training complete!")


Starting model training...


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.302441,0.902000,0.902046


Model training complete!


# Save the Fine-tuned Model and Tokenizer

In [18]:
# Step 3.6: Save the Fine-tuned Model and Tokenizer
output_model_dir = "./fine_tuned_bert_agnews"

print(f"\nSaving fine-tuned model and tokenizer to '{output_model_dir}'...")
# The Trainer's save_model() method saves both the model and its configuration
trainer.save_model(output_model_dir)
# It's also good practice to explicitly save the tokenizer, though it's often saved with the model
tokenizer.save_pretrained(output_model_dir)

print("Fine-tuned model and tokenizer saved successfully!")
print("You can now load this model later using:")
print(f"  from transformers import AutoModelForSequenceClassification, AutoTokenizer")
print(f"  model = AutoModelForSequenceClassification.from_pretrained('{output_model_dir}')")
print(f"  tokenizer = AutoTokenizer.from_pretrained('{output_model_dir}')")


Saving fine-tuned model and tokenizer to './fine_tuned_bert_agnews'...
Fine-tuned model and tokenizer saved successfully!
You can now load this model later using:
  from transformers import AutoModelForSequenceClassification, AutoTokenizer
  model = AutoModelForSequenceClassification.from_pretrained('./fine_tuned_bert_agnews')
  tokenizer = AutoTokenizer.from_pretrained('./fine_tuned_bert_agnews')


# Load the Saved Model and Tokenizer (Optional)

In [19]:
# Step 4.1: Load the Saved Model and Tokenizer (Optional)
from transformers import AutoModelForSequenceClassification, AutoTokenizer

output_model_dir = "./fine_tuned_bert_agnews" # Make sure this matches where you saved it

print(f"\nLoading the fine-tuned model and tokenizer from '{output_model_dir}' for evaluation...")
loaded_model = AutoModelForSequenceClassification.from_pretrained(output_model_dir)
loaded_tokenizer = AutoTokenizer.from_pretrained(output_model_dir)
print("Model and tokenizer loaded successfully for evaluation!")

# Re-initialize the Trainer with the loaded model for final evaluation
# We use the same training_args but only for evaluation purposes
from transformers import Trainer, TrainingArguments, DataCollatorWithPadding
import evaluate # Make sure evaluate is imported if not already
import numpy as np

# Re-define compute_metrics (if you've restarted your session)
accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)
    f1 = f1_metric.compute(predictions=predictions, references=labels, average="weighted")
    return {"accuracy": accuracy["accuracy"], "f1": f1["f1"]}

# A DataCollator is used to form batches by padding sequences to the longest length in that batch.
data_collator = DataCollatorWithPadding(tokenizer=loaded_tokenizer)

# Use dummy training args for evaluation if you don't need to retain the full training config
# Or reuse your existing training_args, ensuring output_dir is where the model is loaded from
eval_training_args = TrainingArguments(
    output_dir="./evaluation_results", # A new temporary directory for evaluation logs
    per_device_eval_batch_size=8,
    report_to="none",
)


trainer = Trainer(
    model=loaded_model,
    args=eval_training_args, # Use args specifically for evaluation
    eval_dataset=tokenized_small_eval_dataset, # Use the prepared evaluation dataset
    tokenizer=loaded_tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)


Loading the fine-tuned model and tokenizer from './fine_tuned_bert_agnews' for evaluation...
Model and tokenizer loaded successfully for evaluation!


/tmp/ipykernel_80/21247911.py:40: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


# Run Final Evaluation

In [20]:
# Step 4.2: Run Final Evaluation
print("\nRunning final evaluation on the evaluation dataset...")
# The trainer already has the loaded_model and tokenized_small_eval_dataset
# from the previous step's re-initialization.
evaluation_results = trainer.evaluate()

print("\n--- Final Evaluation Results ---")
for metric_name, value in evaluation_results.items():
    print(f"{metric_name}: {value:.4f}")
print("------------------------------")


Running final evaluation on the evaluation dataset...


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(



--- Final Evaluation Results ---
eval_loss: 0.3024
eval_accuracy: 0.9020
eval_f1: 0.9020
eval_runtime: 9.3815
eval_samples_per_second: 106.5930
eval_steps_per_second: 6.7150
------------------------------


# Install Gradio and Define a Prediction Function

In [21]:
# Step 5.1: Install Gradio and Define a Prediction Function

# Install Gradio (if not already installed)
try:
    import gradio as gr
except ImportError:
    print("Gradio not found. Installing Gradio...")
    !pip install -q gradio
    import gradio as gr
    print("Gradio installed.")

# Make sure the model and tokenizer are loaded from the saved directory
# If you just ran Step 4.1, they should already be in 'loaded_model' and 'loaded_tokenizer'
# But it's good practice to re-load if you're running cells out of order or after a restart.
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch # Needed for moving tensors to CPU if on GPU

output_model_dir = "./fine_tuned_bert_agnews" # Make sure this matches where you saved it

print(f"\nLoading the fine-tuned model and tokenizer from '{output_model_dir}' for Gradio...")
model_for_inference = AutoModelForSequenceClassification.from_pretrained(output_model_dir)
tokenizer_for_inference = AutoTokenizer.from_pretrained(output_model_dir)
print("Model and tokenizer loaded for inference!")

# Ensure model is on CPU if you're running locally without sufficient VRAM,
# or if you want to explicitly keep it off the GPU for Gradio.
# If running in Colab with GPU, it's fine to keep it on 'cuda'
if torch.cuda.is_available() and model_for_inference.device.type == 'cuda':
    print(f"Model currently on GPU: {model_for_inference.device}. Moving to CPU for consistent inference...")
    model_for_inference.to('cpu')
    print("Model moved to CPU.")


# Define the prediction function for Gradio
def classify_news_topic(text):
    # 1. Tokenize the input text
    inputs = tokenizer_for_inference(text, return_tensors="pt", padding=True, truncation=True)

    # 2. Perform inference
    # Ensure inputs are on the same device as the model (CPU in this case, if moved)
    with torch.no_grad():
        outputs = model_for_inference(**inputs)

    # 3. Get predicted class (logits to probabilities to class ID)
    logits = outputs.logits
    predicted_class_id = torch.argmax(logits, dim=-1).item()

    # 4. Map class ID to human-readable label
    # We need the id2label mapping, which was stored in the model's config
    # You can access it directly from the loaded model
    predicted_label = model_for_inference.config.id2label[predicted_class_id]

    # Optional: Get confidence score
    probabilities = torch.softmax(logits, dim=-1)[0]
    confidence = probabilities[predicted_class_id].item()

    return f"Predicted Topic: {predicted_label} (Confidence: {confidence:.2f})"

print("classify_news_topic function defined.")


Loading the fine-tuned model and tokenizer from './fine_tuned_bert_agnews' for Gradio...
Model and tokenizer loaded for inference!
classify_news_topic function defined.


# Launch the Gradio Interface

In [22]:
# Step 5.2: Launch the Gradio Interface
print("\nLaunching Gradio Interface...")

# Create the Gradio Interface
iface = gr.Interface(
    fn=classify_news_topic,
    inputs=gr.Textbox(lines=5, placeholder="Enter news text here...", label="News Headline/Article"),
    outputs=gr.Textbox(label="Predicted News Topic"),
    title="AG News Topic Classifier",
    description="Enter a news headline or short article to classify its topic (World, Sports, Business, Sci/Tech).",
    live=True, # Update predictions as you type
)

# Launch the interface
# For Kaggle and Colab, `share=True` is necessary to get a public URL
iface.launch(share=True)

print("Gradio Interface launched!")
print("Look for a public URL (e.g., 'https://xxxx.gradio.live') above this cell after execution.")
print("Click on that URL to interact with your model!")


Launching Gradio Interface...
* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://00583c8e770796a0f1.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Gradio Interface launched!
Look for a public URL (e.g., 'https://xxxx.gradio.live') above this cell after execution.
Click on that URL to interact with your model!
